# ★ Least Squares ★

In [27]:
# Import modules
import sys
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import linalg
from scipy import sparse

# 4.1 Least Squares and the normal equations

$$
A^TA\bar{X} = A^Tb
$$

for the least squares solution $\bar{x}$ that minimizes the Euclidean length if the residual $r = b -Ax$

### Example 
Solve the least squares problem 
$$
\begin{bmatrix}
1 & -4 \\ 
2 & 3  \\ 
2 & 2
\end{bmatrix}
\begin{bmatrix}
x_1 \\ 
x_2
\end{bmatrix}
=
\begin{bmatrix}
-3 \\ 
15 \\
9
\end{bmatrix}
$$

In [10]:
A = np.array([1, -4, 2, 3, 2, 2]).reshape(3, 2)
b = np.array([-3, 15, 9])
x = linalg.lstsq(A, b)
print(x[0])

[ 3.8  1.8]


### Example
Find the line that best fits the three data points (t,y)=(1,2),(-1,1) and (1,3) <br/>
The model is $y = c_1 + c_2t$

In [3]:
A = np.array([1, 1, 1, -1, 1, 1]).reshape(3, 2)
b = np.array([2, 1, 3])
x = linalg.lstsq(A, b)
print(x[0])

[ 1.75  0.75]


The best line is $y = \frac{7}{4} + \frac{3}{4}t$ 

# 4.3 QR Factorization

In [81]:
def classical_gram_schmidt_orthogonalization(A):
    Q = np.zeros(A.size).reshape(A.shape)
    R = np.zeros(A.shape[1] ** 2).reshape(A.shape[1], A.shape[1])
    for j in range(A.shape[1]):
        y = A[:,j]
        for i in range(j):
            R[i][j] = np.matmul(Q[:,i], A[:,j])
            y = y - R[i][j] * Q[:,i]
        R[j][j] = linalg.norm(y, 2)
        Q[:,j] = y / R[j][j]
    return Q, R

### Example
Find the reduced QR factorization by applying Gram-Schmidt orthogonalization to the columns of 
$
A = 
\begin{bmatrix}
1 & -4 \\ 
2 & 3  \\ 
2 & 2
\end{bmatrix}
$

In [82]:
A = np.array([1, -4, 2, 3, 2, 2]).reshape(3, 2)
Q, R = classical_gram_schmidt_orthogonalization(A)
print('Q =')
print(Q)
print('R =')
print(R)

Q =
[[ 0.33333333 -0.93333333]
 [ 0.66666667  0.33333333]
 [ 0.66666667  0.13333333]]
R =
[[ 3.  2.]
 [ 0.  5.]]


### Example
Find the full QR factorization of 
$
A = 
\begin{bmatrix}
1 & -4 \\ 
2 & 3  \\ 
2 & 2
\end{bmatrix}
$

In [146]:
A = np.array([1, -4, 2, 3, 2, 2]).reshape(3, 2)
Q, R = linalg.qr(A)
print('Q =')
print(Q)
print('R =')
print(R)

Q =
[[-0.33333333  0.93333333 -0.13333333]
 [-0.66666667 -0.33333333 -0.66666667]
 [-0.66666667 -0.13333333  0.73333333]]
R =
[[-3. -2.]
 [ 0. -5.]
 [ 0.  0.]]


### Example 
Use the full QR factorization to solve the least squares problem
$$
\begin{bmatrix}
1 & -4 \\ 
2 & 3  \\ 
2 & 2
\end{bmatrix}
\begin{bmatrix}
x_1 \\ 
x_2
\end{bmatrix}
=
\begin{bmatrix}
-3 \\ 
15 \\
9
\end{bmatrix}
$$

In [204]:
A = np.array([1, -4, 2, 3, 2, 2]).reshape(3, 2)
b = np.array([-3, 15, 9]).T
Q, R = linalg.qr(A)
lu, piv = linalg.lu_factor(R[:2,:])
x = linalg.lu_solve([lu, piv], np.matmul(Q.T, b).reshape(3, 1)[:2])
print('x = %s' %x.T)

x = [[ 3.8  1.8]]


## Modified Gram-Schmidt orthogonalization

In [3]:
def modified_gram_schmidt_orthogonalization(A):
    Q = np.zeros(A.size).reshape(A.shape)
    R = np.zeros(A.shape[1] ** 2).reshape(A.shape[1], A.shape[1])
    for j in range(A.shape[1]):
        y = A[:,j]
        for i in range(j):
            R[i][j] = np.matmul(Q[:,i], y)
            y = y - R[i][j] * Q[:,i]
        R[j][j] = linalg.norm(y, 2)
        Q[:,j] = y / R[j][j]
    return Q, R

## Householder reflector

### Example

Let $x = [3,4]$ and $w =[5,0]$. Find a householder reflector H that satisfies $Hx = w$.

In [23]:
x = np.array([3, 4]).reshape(2, 1)
w = np.array([5, 0]).reshape(2, 1)
v = w - x

# Projection matrix
P = np.matmul(v, v.T) / np.matmul(v.T, v)

# Householder reflector
H = np.identity(P.shape[0]) - 2 * P

print('H=\n', H)

H=
 [[ 0.6  0.8]
 [ 0.8 -0.6]]


In [104]:
def householder_reflector(x):
    w = np.zeros(x.size)
    w[0] = linalg.norm(x, 2)
    v = (w - x).reshape(x.size, 1)
    
    # Projection matrix
    P = np.matmul(v, v.T) / np.matmul(v.T, v)
    
    # Householder reflector
    H = np.identity(P.shape[0]) - 2 * P
    
    return H

### Example 
Use Householder reflectors to find the QR factorization of 
$A = 
\begin{bmatrix}
3 & 1 \\
4 & 3 \\
\end{bmatrix}
$

In [106]:
A = np.array([3, 1, 4, 3]).reshape(2, 2)
H1 = householder_reflector(A[:,0])
R = np.matmul(H1, A)
Q = H1
print('Q=\n', Q)
print('R=\n', R)

Q=
 [[ 0.6  0.8]
 [ 0.8 -0.6]]
R=
 [[ 5.  3.]
 [ 0. -1.]]


### Example
Use Householder reflectors to find the QR factorization of
$A =
\begin{bmatrix}
1 & -4 \\
2 & 3  \\
2 & 2  \\
\end{bmatrix}
$

In [139]:
A = np.array([1, -4, 2, 3, 2, 2]).reshape(3, 2)
H1 = householder_reflector(A[:, 0])
TEMP = np.matmul(H1, A)
H2 = householder_reflector(TEMP[1:, 1])
H2_Ext = np.identity(H1.shape[0])
H2_Ext[-H2_TMP.shape[0]:, -H2_TMP.shape[1]:] = H2
R = np.matmul(np.matmul(H2_Ext, H1), A)
Q = np.matmul(H1, H2_Ext)
print('Q=\n', Q)
print('R=\n', R)

Q=
 [[ 0.33333333 -0.93333333 -0.13333333]
 [ 0.66666667  0.33333333 -0.66666667]
 [ 0.66666667  0.13333333  0.73333333]]
R=
 [[  3.00000000e+00   2.00000000e+00]
 [ -2.22044605e-16   5.00000000e+00]
 [  0.00000000e+00   0.00000000e+00]]


# 4.4 Generalized Minimum Residual (GMRES) Method

### Example
Solve $Ax = b$ for the following $A$ and $b = [1,0,0]^T$, using GMRES with $x_0 = [0,0,0]^T$

In [28]:
A = np.array([1, 1, 0, 0, 1, 0, 1, 1, 1]).reshape(3, 3)
b = np.array([1, 0, 0]).reshape(3, 1)
x0 = np.zeros(3).reshape(3, 1)
x, info = sparse.linalg.gmres(A, b, x0)
print('x = %s' %x)

x = [ 1.  0. -1.]


In [29]:
A = np.arange(1, 10).reshape(3, 3)
D = np.diag(A.diagonal())
print(D)
print(linalg.inv(D))

[[1 0 0]
 [0 5 0]
 [0 0 9]]
[[ 1.          0.         -0.        ]
 [ 0.          0.2        -0.        ]
 [ 0.          0.          0.11111111]]
